## Questão de negócio

O aprendizado de máquina pode identificar o nível de leitura apropriado de uma passagem de texto e ajudar a inspirar o aprendizado? A leitura é uma habilidade essencial para o sucesso acadêmico. Quando os alunos têm acesso a passagens envolventes que oferecem o nível certo de desafio, eles naturalmente desenvolvem habilidades de leitura.

## Pacotes Python necessários.

* Pandas
* Numpy 
* Sklearn

In [ ]:
# Pacotes necessários
import pandas as pd

## Coleta de dados

**Dataset description**

**Arquivos**
* train.csv
* test.csv

## Pré-processamento de dados

### 1. Limpeza de dados

### 2. Normalização de dados

### 3. Tokenização

### 4. Remoção de palavras irrelevantes

### 5. Vetorização 

## Treinamento do Modelo

## Teste do Modelo

## Gravação do Modelo